In [1]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/app_usage_count_motorola.csv',
            delimiter=';', 
            parse_dates=['date'])
data.head(2)

C:\Users\User\AppData\Local\Temp\ipykernel_10136\727039360.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/app_usage_count_motorola.csv',


,date,Amazon Shopping,Android System,Biblioapp UdeA,Bing,Calculator,Calculator_1,Calendar,Call Management,Camera,...,Twitch,Twitter,tyba,Upwork Talent,VpnDialogs,Wallpapers,WhatsApp,YouTube,YouTube Music,Zoom
0,2022-10-27,0,1,0,0,19,0,1,0,0,...,0,0,0,0,0,0,71,12,0,0
1,2022-10-28,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,100,15,0,0


In [39]:
def transform_dataframe(dataframe: pd.DataFrame, values_greater_than):
    data_copy = dataframe.copy()

    #packing the name of columns and sums in a dictionary to use in further codes
    dictionary_sums = {}
    for column in data_copy:
        if (column != 'date') and \
           (sum(data_copy[column]) > values_greater_than):
            
            dictionary_sums[column] = sum(data_copy[column])

    names_of_columns_to_stay = [column_name for column_name in dictionary_sums.keys()]
    names_of_columns_to_stay.insert(0, 'date')

    #filtering dataframe
    transformed_dataframe = data_copy[names_of_columns_to_stay]
    
    return transformed_dataframe

In [40]:
transform_dataframe(data, 200)

,date,Android System,Calculator,Calendar,Camera_2,Chrome,Clock,Drive,Gmail,Google,...,Spotify,System UI,Tasks,Teams,TikTok,Truecaller,Twitter,WhatsApp,YouTube,YouTube Music
0,2022-10-27,1,19,1,2,30,7,0,3,2,...,3,0,0,0,0,3,0,71,12,0
1,2022-10-28,1,1,0,1,15,3,7,4,2,...,20,0,0,0,0,0,0,100,15,0
2,2022-10-29,0,7,0,1,12,0,0,6,0,...,17,0,0,0,0,2,0,115,22,0
3,2022-10-30,0,2,0,1,13,1,0,2,1,...,6,0,1,0,0,4,0,109,5,0
4,2022-10-31,1,0,0,3,6,2,4,4,4,...,24,0,0,0,0,8,0,104,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023-04-12,4,0,0,5,21,3,0,2,4,...,15,18,8,44,0,3,1,130,3,0
245,2023-04-17,5,2,2,5,18,7,0,1,2,...,17,12,7,31,0,2,1,161,2,0
246,2023-04-21,4,1,3,9,16,7,0,8,3,...,15,11,3,29,0,0,3,112,5,0
247,2023-05-19,3,1,0,9,16,10,0,2,1,...,20,0,11,1,0,0,0,69,1,0


In [37]:
dictionary_sums = {}
for column in data:
    if (column != 'date') and (sum(data[column]) > 200):
        dictionary_sums[column] = sum(data[column])

names_of_columns_to_stay = [column_name for column_name in dictionary_sums.keys()]
names_of_columns_to_stay.insert(0, 'date')

In [38]:
data[names_of_columns_to_stay]

,date,Android System,Calculator,Calendar,Camera_2,Chrome,Clock,Drive,Gmail,Google,...,Spotify,System UI,Tasks,Teams,TikTok,Truecaller,Twitter,WhatsApp,YouTube,YouTube Music
0,2022-10-27,1,19,1,2,30,7,0,3,2,...,3,0,0,0,0,3,0,71,12,0
1,2022-10-28,1,1,0,1,15,3,7,4,2,...,20,0,0,0,0,0,0,100,15,0
2,2022-10-29,0,7,0,1,12,0,0,6,0,...,17,0,0,0,0,2,0,115,22,0
3,2022-10-30,0,2,0,1,13,1,0,2,1,...,6,0,1,0,0,4,0,109,5,0
4,2022-10-31,1,0,0,3,6,2,4,4,4,...,24,0,0,0,0,8,0,104,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023-04-12,4,0,0,5,21,3,0,2,4,...,15,18,8,44,0,3,1,130,3,0
245,2023-04-17,5,2,2,5,18,7,0,1,2,...,17,12,7,31,0,2,1,161,2,0
246,2023-04-21,4,1,3,9,16,7,0,8,3,...,15,11,3,29,0,0,3,112,5,0
247,2023-05-19,3,1,0,9,16,10,0,2,1,...,20,0,11,1,0,0,0,69,1,0
